In [ ]:
#install mlxtend package by opening cmd and
#pip install mlxtend

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [3]:
df = pd.read_csv('france_data.csv')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,22728,ALARM CLOCK BAKELIKE PINK,24,01-12-2010 08:45,3.75,12583.0,France
1,536370,22727,ALARM CLOCK BAKELIKE RED,24,01-12-2010 08:45,3.75,12583.0,France
2,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,01-12-2010 08:45,3.75,12583.0,France
3,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,01-12-2010 08:45,0.85,12583.0,France
4,536370,21883,STARS GIFT TAPE,24,01-12-2010 08:45,0.65,12583.0,France


In [4]:
df.shape

(8408, 8)

In [5]:
#number of unique transactions
len(df.groupby(['InvoiceNo'])['Quantity'].count())

392

In [6]:
#highest billed customer#
max_buy = pd.DataFrame(df.groupby(['CustomerID'])['UnitPrice'].sum().reset_index())
max_buy = max_buy.sort_values(by='UnitPrice', ascending=False)
max_buy.head()

,CustomerID,UnitPrice
16,12536.0,9070.93
55,12681.0,2011.56
56,12682.0,1968.83
20,12567.0,1520.18
48,12669.0,1387.88


In [7]:
#taking a look at the purchases made by customer
highest_cust = df[df['CustomerID']==12536]
highest_cust = highest_cust.sort_values(by='UnitPrice', ascending=False)
highest_cust.head()
#it is seen that there are two products whose Description is Manual and there price is high
#lesson: look at the data carefully before believing it

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
6433,573080,M,Manual,1,27-10-2011 14:20,4161.06,12536.0,France
6432,573077,M,Manual,1,27-10-2011 14:13,4161.06,12536.0,France
6119,570672,22846,BREAD BIN DINER STYLE RED,1,11-10-2011 14:52,16.95,12536.0,France
6120,570672,22847,BREAD BIN DINER STYLE IVORY,1,11-10-2011 14:52,16.95,12536.0,France
6137,570672,23007,SPACEBOY BABY GIFT SET,3,11-10-2011 14:52,16.95,12536.0,France


In [15]:
#second highest billed customer
sec_cust = df[df['CustomerID']==12681]
sec_cust = sec_cust.sort_values(by='UnitPrice', ascending = False)
sec_cust.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
7871,578736,POST,POSTAGE,8,25-11-2011 11:33,18.0,12681.0,France
7516,577121,POST,POSTAGE,4,17-11-2011 18:29,18.0,12681.0,France
7384,576894,POST,POSTAGE,2,17-11-2011 08:35,18.0,12681.0,France
3056,555547,POST,POSTAGE,4,05-06-2011 13:13,18.0,12681.0,France
6914,575193,POST,POSTAGE,5,09-11-2011 09:34,18.0,12681.0,France


In [9]:
#creating a user*product matrix
basket = pd.pivot_table(df,index='InvoiceNo', columns='Description',values='Quantity', fill_value=0)
basket.shape

(392, 1563)

In [10]:
#function to replace natural number with 1
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1) #dropping off the records which have been posted


In [34]:
#applying apriori algorithm to generate rules which have support more than 0.05
frequent_itemsets = apriori(basket_sets, min_support=0.05, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.071429,[4 TRADITIONAL SPINNING TOPS]
1,0.096939,[ALARM CLOCK BAKELIKE GREEN]
2,0.102041,[ALARM CLOCK BAKELIKE PINK]
3,0.094388,[ALARM CLOCK BAKELIKE RED]
4,0.068878,[ASSORTED COLOUR MINI CASES]


In [12]:
#calculating confidence and lift for the genrated rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedants,consequents,support,confidence,lift
0,(PLASTERS IN TIN SPACEBOY),(LUNCH BAG SPACEBOY DESIGN),0.137755,0.388889,3.243499
1,(LUNCH BAG SPACEBOY DESIGN),(PLASTERS IN TIN SPACEBOY),0.119898,0.446809,3.243499
2,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.960000,6.968889
3,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.888889,6.968889
4,(LUNCH BAG WOODLAND),(LUNCH BAG APPLE DESIGN),0.117347,0.456522,3.652174


In [18]:
#selecting only the rules which have lift >6 and confidence >0.8
rules[ (rules['lift'] >= 6) &
       (rules['confidence'] >= 0.8) ]

,antecedants,consequents,support,confidence,lift
2,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.127551,0.960000,6.968889
3,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.137755,0.888889,6.968889
28,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.127551,0.800000,6.030769
32,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE GREEN),0.073980,0.862069,8.892922
33,"(ALARM CLOCK BAKELIKE RED, ALARM CLOCK BAKELIK...",(ALARM CLOCK BAKELIKE PINK),0.079082,0.806452,7.903226
34,"(ALARM CLOCK BAKELIKE GREEN, ALARM CLOCK BAKEL...",(ALARM CLOCK BAKELIKE RED),0.073980,0.862069,9.133271
58,(PACK OF 6 SKULL PAPER PLATES),(PACK OF 6 SKULL PAPER CUPS),0.056122,0.909091,14.254545
59,(PACK OF 6 SKULL PAPER CUPS),(PACK OF 6 SKULL PAPER PLATES),0.063776,0.800000,14.254545
62,(CHILDRENS CUTLERY DOLLY GIRL),(CHILDRENS CUTLERY SPACEBOY),0.071429,0.892857,12.962963
63,(CHILDRENS CUTLERY SPACEBOY),(CHILDRENS CUTLERY DOLLY GIRL),0.068878,0.925926,12.962963


By looking at the rules, it seems that the **green and red alarm clocks are purchased together and the red paper cups, napkins and plates are purchased together** in a manner that is higher than the overall probability would suggest.
At this point, you may want to look at how much opportunity there is to use the popularity of one product to drive sales of another. For instance, we can see that we sell 340 Green Alarm clocks but only 316 Red Alarm Clocks so maybe we can drive more Red Alarm Clock sales through recommendations?

In [33]:
#lets look at a store in german
german = pd.read_csv('german_data.csv')
print(german.shape)
german.head()

(9495, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536527,22809,SET OF 6 T-LIGHTS SANTA,6,01-12-2010 13:04,2.95,12662,Germany
1,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,01-12-2010 13:04,2.55,12662,Germany
2,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,01-12-2010 13:04,0.85,12662,Germany
3,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,01-12-2010 13:04,1.65,12662,Germany
4,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,01-12-2010 13:04,1.95,12662,Germany


In [32]:
basket2 = (german
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules_gen = rules2[ (rules2['lift'] >= 1) &
        (rules2['confidence'] >= 0.5)]
rules_gen.sort_values(by='lift', ascending=False)

,antecedants,consequents,support,confidence,lift
6,(PLASTERS IN TIN CIRCUS PARADE ),(PLASTERS IN TIN WOODLAND ANIMALS),0.087894,0.584906,5.598383
4,(ROUND SNACK BOXES SET OF4 WOODLAND ),(ROUND SNACK BOXES SET OF 4 FRUITS ),0.185738,0.535714,4.486607
5,(ROUND SNACK BOXES SET OF 4 FRUITS ),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.119403,0.833333,4.486607
1,(SPACEBOY LUNCH BOX ),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.077944,0.680851,3.665653
2,(PLASTERS IN TIN WOODLAND ANIMALS),(ROUND SNACK BOXES SET OF4 WOODLAND ),0.104478,0.539683,2.905612


In [ ]:
#you can do analysis on data in different contries and get rules that are interesting